In [ ]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine
pip install pandas
pip install google-cloud-storage


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb**

In [ ]:
import duckdb
import pandas as pd
from google.cloud import storage
from google.colab import auth
auth.authenticate_user()


from google.colab import userdata
import psutil
import gc
from sqlalchemy import create_engine,text
import sys

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////content/dmeyf.db

%sql INSTALL httpfs;
%sql LOAD httpfs;
%sql SET s3_endpoint='storage.googleapis.com';
%sql SET s3_access_key_id='GOOGG3YM7GWPR2PZDRFYO4Y6';
%sql SET s3_secret_access_key='xxx';

,Success


In [ ]:
%%sql
create or replace table datos_temp as
select
    *
from read_csv_auto('s3://buk-tlsssr-dmeyf/datasets/competencia_03_exp_col.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql
create or replace table datos_temp AS
WITH aux AS  (
    SELECT numero_de_cliente, foto_mes,
    (foto_mes//100-2019)*12+foto_mes%100 as mes,
    IF(foto_mes = (select max(foto_mes) from competencia_03) ,
    -1, lead(mes,1,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+1',
    IF(foto_mes = (select max(foto_mes) from competencia_03) OR foto_mes = ((select max(foto_mes) from competencia_03)-1),
     -1,lead(mes,2,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+2',
    FROM
       competencia_03
    ORDER BY numero_de_cliente, foto_mes)
SELECT *,
CASE
    -- Condiciones de borde:
    WHEN ("mes+1"==-1) THEN NULL
    WHEN ("mes+2"==-1) THEN NULL
    -- Si mes siguiente no es consecutivo o no existe:
    WHEN ("mes"+1 != "mes+1") THEN 'BAJA+1'
    WHEN ("mes"+2 != "mes+2") THEN 'BAJA+2'
    -- si proximos 2 meses consecutivos activos:
    ELSE 'CONTINUA' END AS "clase_ternaria"
FROM aux
LEFT JOIN competencia_03 c USING (numero_de_cliente, foto_mes);
commit;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
def consulta_sql(consulta):
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      db.execute(text(consulta))
      db.commit()
      db.close()
      engine.dispose()
      gc.collect()
    print(f"Ejecutado: {consulta}")

In [ ]:
# Variables null tarjetas a cero
engine = create_engine("duckdb:////content/dmeyf.db")

with engine.connect() as db:
  consulta = f"SELECT column_name FROM information_schema.columns WHERE table_name = 'datos_temp';"
  columnas =  list(db.execute(text(consulta)).fetchall())
columnas = [r for r, in columnas]
db.close()
engine.dispose()
gc.collect()

for col in columnas:
    if col.startswith("Visa_") or col.startswith("Master_"):
        consulta = f"update datos_temp set {col} = 0 where {col} is null;"
        consulta_sql(consulta)


otras = ['mtarjeta_master_descuentos','mtarjeta_visa_descuentos','cmobile_app_trx']
for col in otras:
      consulta = f"update datos_temp set {col} = 0 where {col} is null;"
      consulta_sql(consulta)


Ejecutado: update datos_temp set Master_delinquency = 0 where Master_delinquency is null;
Ejecutado: update datos_temp set Master_status = 0 where Master_status is null;
Ejecutado: update datos_temp set Master_mfinanciacion_limite = 0 where Master_mfinanciacion_limite is null;
Ejecutado: update datos_temp set Master_Fvencimiento = 0 where Master_Fvencimiento is null;
Ejecutado: update datos_temp set Master_Finiciomora = 0 where Master_Finiciomora is null;
Ejecutado: update datos_temp set Master_msaldototal = 0 where Master_msaldototal is null;
Ejecutado: update datos_temp set Master_msaldopesos = 0 where Master_msaldopesos is null;
Ejecutado: update datos_temp set Master_msaldodolares = 0 where Master_msaldodolares is null;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Ejecutado: update datos_temp set Master_mconsumospesos = 0 where Master_mconsumospesos is null;
Ejecutado: update datos_temp set Master_mconsumosdolares = 0 where Master_mconsumosdolares is null;
Ejecutado: update datos_temp set Master_mlimitecompra = 0 where Master_mlimitecompra is null;
Ejecutado: update datos_temp set Master_madelantopesos = 0 where Master_madelantopesos is null;
Ejecutado: update datos_temp set Master_madelantodolares = 0 where Master_madelantodolares is null;
Ejecutado: update datos_temp set Master_fultimo_cierre = 0 where Master_fultimo_cierre is null;
Ejecutado: update datos_temp set Master_mpagado = 0 where Master_mpagado is null;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Ejecutado: update datos_temp set Master_mpagospesos = 0 where Master_mpagospesos is null;
Ejecutado: update datos_temp set Master_mpagosdolares = 0 where Master_mpagosdolares is null;
Ejecutado: update datos_temp set Master_fechaalta = 0 where Master_fechaalta is null;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Ejecutado: update datos_temp set Master_mconsumototal = 0 where Master_mconsumototal is null;
Ejecutado: update datos_temp set Master_cconsumos = 0 where Master_cconsumos is null;
Ejecutado: update datos_temp set Master_cadelantosefectivo = 0 where Master_cadelantosefectivo is null;
Ejecutado: update datos_temp set Master_mpagominimo = 0 where Master_mpagominimo is null;
Ejecutado: update datos_temp set Visa_delinquency = 0 where Visa_delinquency is null;
Ejecutado: update datos_temp set Visa_status = 0 where Visa_status is null;
Ejecutado: update datos_temp set Visa_mfinanciacion_limite = 0 where Visa_mfinanciacion_limite is null;
Ejecutado: update datos_temp set Visa_Fvencimiento = 0 where Visa_Fvencimiento is null;
Ejecutado: update datos_temp set Visa_Finiciomora = 0 where Visa_Finiciomora is null;
Ejecutado: update datos_temp set Visa_msaldototal = 0 where Visa_msaldototal is null;
Ejecutado: update datos_temp set Visa_msaldopesos = 0 where Visa_msaldopesos is null;
Ejecutado: upd

In [ ]:
%%sql
UPDATE datos_temp SET cmobile_app_trx = CAST(cmobile_app_trx AS INTEGER);
UPDATE datos_temp SET tmobile_app = CAST(tmobile_app AS INTEGER);
ALTER TABLE datos_temp ALTER COLUMN cmobile_app_trx SET DATA TYPE INTEGER;
ALTER TABLE datos_temp ALTER COLUMN tmobile_app SET DATA TYPE INTEGER;
create or replace table datos_temp_bk  as select * from datos_temp;
commit;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
# interpolación
from sqlalchemy.pool import reset_commit

engine = create_engine("duckdb:////content/dmeyf.db")
with engine.connect() as conn:
  consulta = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'datos_temp';"
  tipos =  dict(conn.execute(text(consulta)).fetchall())
  columnas = tipos.keys()
engine.dispose()
tabla = 'datos_temp'
conn.close()
engine.dispose()
gc.collect()

for col in columnas:
  if not col.startswith("ct_0_") and col != "clase_ternaria":
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      consulta = f"SELECT sum({col}), foto_mes  FROM datos_temp group by foto_mes;"
      res =  list(db.execute(text(consulta)).fetchall())
      for sum, mes in res:
        if sum == 0:
          try:
            db.execute(text(f"ALTER TABLE {tabla} RENAME {col} TO ct_0_{col};"))
            print(f"creado ct_0_{col}")
          except:
            pass
          db.commit()
    db.close()
    engine.dispose()
    gc.collect()

engine = create_engine("duckdb:////content/dmeyf.db")
with engine.connect() as conn:
  consulta = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'datos_temp';"
  tipos =  dict(conn.execute(text(consulta)).fetchall())
  columnas = tipos.keys()
engine.dispose()
db.close()
engine.dispose()
gc.collect()

for col in columnas:
  if col.startswith("ct_0_") and col != "clase_ternaria":
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      consulta = f"SELECT sum({col}), foto_mes  FROM datos_temp group by foto_mes;"
      res =  list(db.execute(text(consulta)).fetchall())
      for sum, mes in res:
        if sum == 0:
          col_name = f"ct_it_{col[5:]}"
          try:
            db.execute(text(f"ALTER TABLE datos_temp ADD COLUMN {col_name} {tipos[col]};"))
            db.execute(text(f"UPDATE datos_temp SET {col_name} = {col};"))
          except:
            pass
          db.commit()
          consulta = f"""WITH aux AS
                        (
                            SELECT
                            foto_mes, numero_de_cliente,
                            (IF({col}=0,
                            (LEAD({col}) OVER (ORDER BY numero_de_cliente, foto_mes) + LAG({col}) OVER (ORDER BY numero_de_cliente, foto_mes)) / 2,
                            {col} ))
                            AS nuevo_valor

                            FROM
                                datos_temp
                            ORDER BY
                                numero_de_cliente, foto_mes
                        )
                        UPDATE datos_temp set {col_name} = aux.nuevo_valor
                        FROM aux
                        WHERE datos_temp.foto_mes = aux.foto_mes
                        and datos_temp.numero_de_cliente = aux.numero_de_cliente
                        and datos_temp.foto_mes={mes};"""
          db.execute(text(consulta))
          print(f"actualizado {col} {mes}")
          db.commit()
    db.close()
    engine.dispose()
    gc.collect()

creado ct_0_active_quarter
creado ct_0_internet
creado ct_0_mrentabilidad
creado ct_0_mrentabilidad_annual
creado ct_0_mcomisiones
creado ct_0_mactivos_margen
creado ct_0_mpasivos_margen
creado ct_0_mcuentas_saldo
creado ct_0_ctarjeta_debito_transacciones
creado ct_0_mautoservicio
creado ct_0_ctarjeta_visa_transacciones
creado ct_0_mtarjeta_visa_consumo
creado ct_0_ctarjeta_master_transacciones
creado ct_0_mtarjeta_master_consumo
creado ct_0_ctarjeta_visa_debitos_automaticos
creado ct_0_mttarjeta_visa_debitos_automaticos
creado ct_0_ccajeros_propios_descuentos
creado ct_0_mcajeros_propios_descuentos
creado ct_0_ctarjeta_visa_descuentos
creado ct_0_mtarjeta_visa_descuentos
creado ct_0_ctarjeta_master_descuentos
creado ct_0_mtarjeta_master_descuentos
creado ct_0_ccomisiones_otras
creado ct_0_mcomisiones_otras
creado ct_0_cextraccion_autoservicio
creado ct_0_mextraccion_autoservicio
creado ct_0_ccheques_depositados
creado ct_0_mcheques_depositados
creado ct_0_ccheques_emitidos
creado ct_0

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_0_active_quarter 202006


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_0_internet 202006


...

actualizado ct_0_matm_other 202006


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_0_tmobile_app 202006


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_0_cmobile_app_trx 202006


In [ ]:
# media
engine = create_engine("duckdb:////content/dmeyf.db")
with engine.connect() as conn:
  consulta = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'datos_temp';"
  tipos =  dict(conn.execute(text(consulta)).fetchall())
  columnas = tipos.keys()
engine.dispose()
conn.close()
engine.dispose()
gc.collect()

for col in columnas:
  if col.startswith("ct_0_") and col != "clase_ternaria":
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      consulta = f"SELECT sum({col}), foto_mes  FROM datos_temp group by foto_mes;"
      res =  list(db.execute(text(consulta)).fetchall())
      for sum, mes in res:
        if sum == 0:
          col_name = f"ct_mean_6m_{col[5:]}"
          try:
            db.execute(text(f"ALTER TABLE datos_temp ADD COLUMN {col_name} {tipos[col]};"))
            db.execute(text(f"UPDATE datos_temp SET {col_name} = {col};"))
          except:
            pass
          consulta = f"""WITH aux AS
                        (
                            SELECT
                                foto_mes, numero_de_cliente,
                                AVG({col}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS nuevo_valor
                            FROM
                                datos_temp
                            ORDER BY
                                numero_de_cliente, foto_mes
                        )
                        UPDATE datos_temp set {col_name} = aux.nuevo_valor
                        FROM aux
                        WHERE datos_temp.foto_mes = aux.foto_mes
                        and datos_temp.numero_de_cliente = aux.numero_de_cliente
                        and datos_temp.foto_mes={mes};"""
          db.execute(text(consulta))
          print(f"actualizado {col_name} {col} {mes}")
          db.commit()
    db.close()
    engine.dispose()
    gc.collect()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_mean_6m_active_quarter ct_0_active_quarter 202006


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_mean_6m_internet ct_0_internet 202006


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

actualizado ct_mean_6m_matm_other ct_0_matm_other 202006


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_mean_6m_tmobile_app ct_0_tmobile_app 202006


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_mean_6m_cmobile_app_trx ct_0_cmobile_app_trx 202006


In [ ]:
# Mediana
engine = create_engine("duckdb:////content/dmeyf.db")
with engine.connect() as conn:
  consulta = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'datos_temp';"
  tipos =  dict(conn.execute(text(consulta)).fetchall())
  columnas = tipos.keys()
engine.dispose()
conn.close()
engine.dispose()
gc.collect()

for i, col in enumerate(columnas):
  print(f"{i}/{len(columnas)}")
  if col.startswith("ct_0_") and col != "clase_ternaria":
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      consulta = f"SELECT sum({col}), foto_mes  FROM datos_temp group by foto_mes;"
      res =  list(db.execute(text(consulta)).fetchall())
      for sum, mes in res:
        if sum == 0:
          col_name = f"ct_median_6m_{col[5:]}"
          try:
            db.execute(text(f"ALTER TABLE datos_temp ADD COLUMN {col_name} {tipos[col]};"))
            db.execute(text(f"UPDATE datos_temp SET {col_name} = {col};"))
          except:
            pass
          consulta = f"""WITH aux AS
                        (
                            SELECT
                                foto_mes, numero_de_cliente,
                                median({col}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS nuevo_valor
                            FROM
                                datos_temp
                            ORDER BY
                                numero_de_cliente, foto_mes
                        )
                        UPDATE datos_temp set {col_name} = aux.nuevo_valor
                        FROM aux
                        WHERE datos_temp.foto_mes = aux.foto_mes
                        and datos_temp.numero_de_cliente = aux.numero_de_cliente
                        and datos_temp.foto_mes={mes};"""
          db.execute(text(consulta))
          print(f"actualizado {col_name} {col} {mes}")
          db.commit()
    db.close()
    engine.dispose()
    gc.collect()

0/354
1/354
2/354
3/354
4/354
5/354


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_median_6m_active_quarter ct_0_active_quarter 202006
6/354
7/354


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_median_6m_internet ct_0_internet 202006
8/354
9/354
10/354


...

actualizado ct_median_6m_matm_other ct_0_matm_other 202006
110/354
111/354


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_median_6m_tmobile_app ct_0_tmobile_app 202006
112/354


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

actualizado ct_median_6m_cmobile_app_trx ct_0_cmobile_app_trx 202006
113/354
114/354
115/354
116/354
117/354
118/354
119/354
120/354
121/354
122/354
123/354
124/354
125/354
126/354
127/354
128/354
129/354
130/354
131/354
132/354
133/354
134/354
135/354
136/354
137/354
138/354
139/354
140/354
141/354
142/354
143/354
144/354
145/354
146/354
147/354
148/354
149/354
150/354
151/354
152/354
153/354
154/354
155/354
156/354
157/354
158/354
159/354
160/354
161/354
162/354
163/354
164/354
165/354
166/354
167/354
168/354
169/354
170/354
171/354
172/354
173/354
174/354
175/354
176/354
177/354
178/354
179/354
180/354
181/354
182/354
183/354
184/354
185/354
186/354
187/354
188/354
189/354
190/354
191/354
192/354
193/354
194/354
195/354
196/354
197/354
198/354
199/354
200/354
201/354
202/354
203/354
204/354
205/354
206/354
207/354
208/354
209/354
210/354
211/354
212/354
213/354
214/354
215/354
216/354
217/354
218/354
219/354
220/354
221/354
222/354
223/354
224/354
225/354
226/354
227/354
228/354
229

In [ ]:
# nulls
engine = create_engine("duckdb:////content/dmeyf.db")
with engine.connect() as conn:
  consulta = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'datos_temp';"
  tipos =  dict(conn.execute(text(consulta)).fetchall())
  columnas = tipos.keys()
engine.dispose()
conn.close()
engine.dispose()
gc.collect()

for i, col in enumerate(columnas):
  print(f"{i}/{len(columnas)}")
  if col.startswith("ct_0_") and col != "clase_ternaria":
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      consulta = f"SELECT sum({col}), foto_mes  FROM datos_temp group by foto_mes;"
      res =  list(db.execute(text(consulta)).fetchall())
      for sum, mes in res:
        if sum == 0:
          col_name = f"ct_null_{col[5:]}"
          try:
            db.execute(text(f"ALTER TABLE datos_temp ADD COLUMN {col_name} {tipos[col]};"))
            db.execute(text(f"UPDATE datos_temp SET {col_name} = {col};"))
          except:
            pass
          consulta = f"""
                        UPDATE datos_temp set {col_name} = NULL
                        WHERE datos_temp.foto_mes={mes};"""
          db.execute(text(consulta))
          print(f"actualizado {col_name} {col} {mes}")
          db.commit()
    db.close()
    engine.dispose()
    gc.collect()

0/404
1/404
2/404
3/404
4/404
5/404
actualizado ct_null_active_quarter ct_0_active_quarter 202006
6/404
7/404
actualizado ct_null_internet ct_0_internet 202006
8/404
9/404
10/404
actualizado ct_null_mrentabilidad ct_0_mrentabilidad 201905
actualizado ct_null_mrentabilidad ct_0_mrentabilidad 201910
actualizado ct_null_mrentabilidad ct_0_mrentabilidad 202006
11/404
actualizado ct_null_mrentabilidad_annual ct_0_mrentabilidad_annual 201905
actualizado ct_null_mrentabilidad_annual ct_0_mrentabilidad_annual 201910
actualizado ct_null_mrentabilidad_annual ct_0_mrentabilidad_annual 202006
12/404
actualizado ct_null_mcomisiones ct_0_mcomisiones 201905
actualizado ct_null_mcomisiones ct_0_mcomisiones 201910
actualizado ct_null_mcomisiones ct_0_mcomisiones 202006
13/404
actualizado ct_null_mactivos_margen ct_0_mactivos_margen 201905
actualizado ct_null_mactivos_margen ct_0_mactivos_margen 201910
actualizado ct_null_mactivos_margen ct_0_mactivos_margen 202006
14/404
actualizado ct_null_mpasivos_ma

In [ ]:
def add_historicos(tabla, col, cant, tipo_col):
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      consulta = f"SELECT lag_{col}_{cant} FROM {tabla} LIMIT 1;"
      try:
        db.execute(text(consulta)).fetchone()
      except:
        print(f"add historicos: {col} {cant}")
        cant = [cant]
        for x in cant:
          consulta = f"ALTER TABLE {tabla} ADD COLUMN {col}_{x} {tipo_col};"
          try:
            db.execute(text(consulta))
          except:
            pass
        db.commit()
        n = cant
        particiones =""
        for i in cant:
            particiones += f"IFNULL(LAG({col}, {i}) OVER (PARTITION BY numero_de_cliente ORDER BY mes), -1) AS {col}_{i}\n"

        operaciones=""
        for i in cant:
            operaciones += f"{col}_{i} = t.{col}_{i}\n"

        consulta = f"""
        WITH t AS
        (SELECT
        numero_de_cliente,
        mes,
        {col},
        {particiones}
        FROM {tabla})
        UPDATE {tabla} SET
        {operaciones}
        FROM t
        WHERE ({tabla}.numero_de_cliente = t.numero_de_cliente AND {tabla}.mes = t.mes);
        """
        db.execute(text(consulta))
        db.execute(text(f"ALTER TABLE {tabla} RENAME {col}_{cant[0]} TO lag_{col}_{cant[0]};"))
      else:
        print(f"{col} ya creada")
        return
      finally:
        db.commit()
        db.close()
        engine.dispose()
        gc.collect()


def eliminar_columna(tabla, col):
    engine = create_engine("duckdb:////content/dmeyf.db")
    with engine.connect() as db:
      consulta = f"SELECT {col} FROM {tabla} LIMIT 1;"
      try:
        db.execute(text(consulta)).fetchone()
      except Exception as e:
        print(f"{col} no existe")
        return
      print(f"Eliminando: {col}")
      db.execute(text(f"ALTER TABLE '{tabla}' DROP COLUMN {col};"))
    engine.dispose()
def memoria_variables(variables_locales, variables_globales):
    variable_sizes = [(nombre, sys.getsizeof(variable)) for nombre, variable in variables_locales.items()]
    variable_sizes.sort(key=lambda x: x[1], reverse=True)
    for i, (nombre, tamaño) in enumerate(variable_sizes[:3]):
        print(f"Variable local #{i+1}: '{nombre}' ocupa {tamaño} bytes en memoria")

    variable_sizes = [(nombre, sys.getsizeof(variable)) for nombre, variable in variables_globales.items()]
    variable_sizes.sort(key=lambda x: x[1], reverse=True)
    for i, (nombre, tamaño) in enumerate(variable_sizes[:3]):
        print(f"Variable global #{i+1}: '{nombre}' ocupa {tamaño} bytes en memoria")

from psutil._common import bytes2human

def memory(num):
  mem = psutil.virtual_memory()
  # Convierte la instancia de svmem en un diccionario
  #mem_dict = {
      #"total": bytes2human(mem.total),
      #"available": bytes2human(mem.available),
      #"percent": bytes2human(mem.percent),
      #"used": bytes2human(mem.used),
      #"free": bytes2human(mem.free),
      #"active": bytes2human(mem.active),
      #"inactive": bytes2human(mem.inactive),
      #"buffers": bytes2human(mem.buffers),
      #"cached": bytes2human(mem.cached),
      #"shared": bytes2human(mem.shared),
      #"slab": bytes2human(mem.slab),}

  print(f"{num} - inactive mem: {bytes2human(mem.inactive)}")

In [ ]:
# LAGS
import psutil
import gc

from sqlalchemy import create_engine,text
engine = create_engine("duckdb:////content/dmeyf.db")

with engine.connect() as conn:
  consulta = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'datos_temp';"
  columnas =  dict(conn.execute(text(consulta)).fetchall())
engine.dispose()
tabla = 'datos_temp'

conn.close()
gc.collect()


for i, col in enumerate(columnas):
  print(f'{i}/{len(columnas)}')
  if col not in ['numero_de_cliente', 'foto_mes','mes','mes+1','mes+2'] and not col.startswith("lag_") and col.startswith("ct_null"):
    add_historicos(tabla,col,3,columnas[col])
    gc.collect()
    add_historicos(tabla,col,6,columnas[col])
    gc.collect()
    add_historicos(tabla,col,12,columnas[col])
    gc.collect()


0/601
1/601
2/601
3/601
4/601
5/601
6/601
7/601
8/601
9/601
10/601
11/601
12/601
13/601
14/601
15/601
16/601
17/601
18/601
19/601
20/601
21/601
22/601
23/601
24/601
25/601
26/601
27/601
28/601
29/601
30/601
31/601
32/601
33/601
34/601
35/601
36/601
37/601
38/601
39/601
40/601
41/601
42/601
43/601
44/601
45/601
46/601
47/601
48/601
49/601
50/601
51/601
52/601
53/601
54/601
55/601
56/601
57/601
58/601
59/601
60/601
61/601
62/601
63/601
64/601
65/601
66/601
67/601
68/601
69/601
70/601
71/601
72/601
73/601
74/601
75/601
76/601
77/601
78/601
79/601
80/601
81/601
82/601
83/601
84/601
85/601
86/601
87/601
88/601
89/601
90/601
91/601
92/601
93/601
94/601
95/601
96/601
97/601
98/601
99/601
100/601
101/601
102/601
103/601
104/601
105/601
106/601
107/601
108/601
109/601
110/601
111/601
112/601
113/601
114/601
115/601
116/601
117/601
118/601
119/601
120/601
121/601
122/601
123/601
124/601
125/601
126/601
127/601
128/601
129/601
130/601
131/601
132/601
133/601
134/601
135/601
136/601
137/601
138/60

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

add historicos: ct_null_mautoservicio_6_3 6


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

add historicos: ct_null_mautoservicio_6_3 12


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
eliminar = [
  # "numero_de_cliente",
   # "foto_mes",
    "cforex_buy",
    "cforex_sell",
    "Master_Fvencimiento",
    "Master_fultimo_cierre",
    "Visa_Fvencimiento",
    "Visa_fultimo_cierre",
]
for col in eliminar:
  eliminar_columna(tabla, col)

Eliminando: cforex_buy
Eliminando: cforex_sell
Eliminando: Master_Fvencimiento
Eliminando: Master_fultimo_cierre
Eliminando: Visa_Fvencimiento
Eliminando: Visa_fultimo_cierre


In [ ]:
%%sql
ALTER TABLE datos_temp DROP COLUMN "mes+1";
ALTER TABLE datos_temp DROP COLUMN "mes+2";
commit;

RuntimeError: (duckdb.BinderException) Binder Error: Table "datos_temp" does not have a column with name "mes+1"
[SQL: ALTER TABLE datos_temp DROP COLUMN "mes+1";]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
import psutil
import gc


#conn = duckdb.connect('dmeyf.db')
from sqlalchemy import create_engine,text
engine = create_engine("duckdb:////content/dmeyf.db")

with engine.connect() as conn:
  consulta = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'datos_temp';"
  columnas =  dict(conn.execute(text(consulta)).fetchall())
engine.dispose()
tabla = 'datos_temp'

conn.close()
gc.collect()

columnas_lag = []
for i, col in enumerate(columnas):
  if col.startswith("ct_null") or col.startswith("lag_ct_null"):
    columnas_lag.append(col)

with engine.connect() as conn:
  select_query = f"COPY (SELECT {', '.join(columnas_lag)} FROM datos_temp) TO 'competencia_03_exp_col_4.csv.gz' (FORMAT CSV, HEADER 1);"
  result_select = conn.execute(text(select_query))
conn.close()
engine.dispose()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
from sqlalchemy import create_engine,text
engine = create_engine("duckdb:////content/dmeyf.db")

with engine.connect() as conn:
  query = """
  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'datos_temp' AND COLUMN_NAME NOT LIKE 'lag_%'
  """
  result = conn.execute(text(query))

  # Obtener las columnas como una lista
  columnas_lag = [row[0] for row in result.fetchall()]
  print(columnas_lag)
  # Construir la consulta SELECT
  select_query = f"COPY (SELECT {', '.join(columnas_lag)} FROM datos_temp) TO 'competencia_03_exp_col.csv.gz' (FORMAT CSV, HEADER 1);"

  # Ejecutar la consulta SELECT
  result_select = conn.execute(text(select_query))
  # Cerrar la conexión
conn.close()
engine.dispose()

['numero_de_cliente', 'foto_mes', 'mes', 'mes+1', 'mes+2', 'ct_0_active_quarter', 'cliente_vip', 'ct_0_internet', 'cliente_edad', 'cliente_antiguedad', 'ct_0_mrentabilidad', 'ct_0_mrentabilidad_annual', 'ct_0_mcomisiones', 'ct_0_mactivos_margen', 'ct_0_mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'ct_0_mcuentas_saldo', 'ctarjeta_debito', 'ct_0_ctarjeta_debito_transacciones', 'ct_0_mautoservicio', 'ctarjeta_visa', 'ct_0_ctarjeta_visa_transacciones', 'ct_0_mtarjeta_visa_consumo', 'ctarjeta_master', 'ct_0_ctarjeta_master_transacciones', 'ct_0_mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
from sqlalchemy import create_engine,text
engine = create_engine("duckdb:////content/dmeyf.db")

with engine.connect() as conn:
  query = """
  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'datos_temp' AND COLUMN_NAME NOT LIKE 'lag_ct_m%' AND COLUMN_NAME LIKE 'lag%'
  """
  result = conn.execute(text(query))

  # Obtener las columnas como una lista
  columnas_lag = [row[0] for row in result.fetchall()]
  print(columnas_lag)
  # Construir la consulta SELECT
  select_query = f"COPY (SELECT {', '.join(columnas_lag)} FROM datos_temp) TO 'competencia_03_exp_col_lags_1.csv.gz' (FORMAT CSV, HEADER 1);"

  # Ejecutar la consulta SELECT
  result_select = conn.execute(text(select_query))
  # Cerrar la conexión
conn.close()
engine.dispose()

['lag_ct_0_active_quarter_3', 'lag_ct_0_active_quarter_6', 'lag_ct_0_active_quarter_12', 'lag_cliente_vip_3', 'lag_cliente_vip_6', 'lag_cliente_vip_12', 'lag_ct_0_internet_3', 'lag_ct_0_internet_6', 'lag_ct_0_internet_12', 'lag_cliente_edad_3', 'lag_cliente_edad_6', 'lag_cliente_edad_12', 'lag_cliente_antiguedad_3', 'lag_cliente_antiguedad_6', 'lag_cliente_antiguedad_12', 'lag_ct_0_mrentabilidad_3', 'lag_ct_0_mrentabilidad_6', 'lag_ct_0_mrentabilidad_12', 'lag_ct_0_mrentabilidad_annual_3', 'lag_ct_0_mrentabilidad_annual_6', 'lag_ct_0_mrentabilidad_annual_12', 'lag_ct_0_mcomisiones_3', 'lag_ct_0_mcomisiones_6', 'lag_ct_0_mcomisiones_12', 'lag_ct_0_mactivos_margen_3', 'lag_ct_0_mactivos_margen_6', 'lag_ct_0_mactivos_margen_12', 'lag_ct_0_mpasivos_margen_3', 'lag_ct_0_mpasivos_margen_6', 'lag_ct_0_mpasivos_margen_12', 'lag_cproductos_3', 'lag_cproductos_6', 'lag_cproductos_12', 'lag_tcuentas_3', 'lag_tcuentas_6', 'lag_tcuentas_12', 'lag_ccuenta_corriente_3', 'lag_ccuenta_corriente_6', 'la

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp competencia_03_exp_col_4.csv.gz /content/drive/MyDrive

Mounted at /content/drive
